In [ ]:
require 'torch'
require 'nn'
require 'optim'

In [ ]:
hdf5 = require 'hdf5'

In [ ]:
file = hdf5.open("../data/prepped_train_X_word2vec_wordvecs.h5", 'r')
data = file:read():all()
file:close()

In [ ]:
y_csv = csvigo.load({path = "../data/prepped_train_y.csv",
    verbose = false, mode = "raw"})

In [ ]:
X_in = data["dataset_1"]

In [ ]:
-- remove column names
table.remove(y_csv, 1)

-- this is important
y_in = torch.Tensor(y_csv)[{ {}, {2,9} }]

In [ ]:
classes = {'1', '2', '3', '4', '5', '6', '7', '8'}

In [ ]:
-- TODO: load actual word vectors
-- first dimension: 150 examples
-- second dimension: 200 words OR 8 labels
-- third dimension: word vectors of dimensionality 300
train_inputs = X_in
train_targets = y_in

## Data
Input format is a matrix $M \in \mathbb{R}^{n} \times k$
with:
* $n$ number of words in the document
* $k$ the dimension of the word vectors obtained from word2vec
* the data is zero-padded to the length of the longest document
(alternatively to a given maximum length). All not-taken word spaces
are set to zero in the matrix.

## Hyperparameters
* Filter windows of height $h$ are applied in sizes of $3,4,5$ with TODO feature
maps. Specifically the filters are applied as a matrix $W \in \mathbb{R}^{h \times k}$
to a window of size $h \times k$.
* The stride of the filters is $1$ to iterate over all possible windows of words
(*narrow convolution*).
* A dropout of rate $\rho = .5$ is applied at training time to prevent overfitting.
* An $l_2$ constraint $s = 3$ is applied in (Kim, 2014). However (Zhang and Wallace, 2015)
found that this constraint had little effect on the performance of the network.

In [ ]:
-- metadata
batch_size = 100
no_examples = train_inputs:size()[1]
no_classes = train_targets:size()[2]
sent_len = 150
wordvec_len = 300
dropout_rho = .5
l2_constr = 3

In [ ]:
-- we have to copy the matrix 6 times to use the parallel container
-- also we squash the words to a given length, if not already done so.
-- zero padding is applied as well.
X = train_inputs
y = train_targets

-- FIXME: this is way too complex and memory-inefficient. should use storage
-- and stuff in the future.
function copy_example(example)
    local X_i = torch.Tensor(6, sent_len, wordvec_len):zero()

    for j = 1, sent_len do
        X_i[{1, j}] = example[j]
        X_i[{2, j}] = example[j]
        X_i[{3, j}] = example[j]
        X_i[{4, j}] = example[j]
        X_i[{5, j}] = example[j]
        X_i[{6, j}] = example[j]
    end

    return X_i
end

## Model

In [62]:
model = nn.Sequential()
model:add(nn.Identity())

-- stage 1: convolutions

-- in: applies the following declared layers to the first dimension of the
-- tensor (see above.)
--
-- out: concatenating the three concatenated max-pooled values to a vector
-- fed to the fully connected softmax layer yielding the outputs

-- in: tensor of 6 x num_exaples x sent_len x wordvec_len
--p = nn.ConcatTable()
p = nn.Parallel(1,1)

-- this is a convolution unit differing in the height of the filter being
-- applied. each filter is used double time to further improve performance.
-- each filter yields a feature map, thus for each region size we then
-- have two feature maps, and in total then six (if using default n-grams
-- 3 to 5 like (Kim, 2014))
for i = 3, 5 do
    -- elements of the convolution
    local s1 = nn.Sequential()
    local s2 = nn.Sequential()

    -- input: sent_len x wordvec_len
    s1:add(nn.Reshape(1, sent_len, wordvec_len))
    s2:add(nn.Reshape(1, sent_len, wordvec_len))

    -- takes size of input plane (we only have one channel though)
    -- as well as output plane (again, using only one channel)
    -- kernel width and kernel height as third and fourth arguments

    -- in: 1 x sent_len x wordvec_len
    -- args: input channels, output channels, kernel width, kernel height
    s1:add(nn.SpatialConvolution(1, 1, wordvec_len, i))
    s2:add(nn.SpatialConvolution(1, 1, wordvec_len, i))
    -- out: sent_len - filter_height + 1

    -- non-linearities
    -- in: sent_len - filter_height + 1
    s1:add(nn.ReLU())
    s2:add(nn.ReLU())
    -- out: sent_len - filter_height + 1

    -- the viewed region of the matrix for max-pooling shall be the
    -- size of the matrix, as we want all values to be considered at
    -- once for a single maximum for each filter map.
    -- in: sent_len - filter_height + 1
    s1:add(nn.SpatialMaxPooling(1, sent_len - i + 1))
    s2:add(nn.SpatialMaxPooling(1, sent_len - i + 1))
    -- out: scalar

    p:add(s1)
    p:add(s2)
end

model:add(p)

-- [6x1x1] -> [1x6]
model:add(nn.View(1,6))

-- stage 2: fully connected softmax layer
model:add(nn.Normalize(2, l2_constr))
model:add(nn.Dropout(dropout_rho))
model:add(nn.Linear(6, 8))

--model:add(nn.LogSoftMax()) -- for ClassNLLCriterion
model:add(nn.Sigmoid())

## Training
The network is trained with Stochastic Gradient Descent (SGD)
with randomly shuffled mini-batches and the Adadelta update rule (Zeiler, 2012).

In [130]:
function next_batch(offset)
    end_offset = offset + batch_size

    if end_offset > X:size()[2] then
        end_offset = offset + (X:size()[2] - offset)
    end

    return X[{{offset, end_offset}, {}, {}}], y[{{offset, end_offset}, {}}]
end

In [131]:
criterion = nn.BCECriterion()

In [132]:
parameters, grad_parameters = model:getParameters()
model:training()

In [118]:
for i = 1, no_examples, batch_size do
    inputs, targets = next_batch(i)
    num_inputs = inputs:size()[2]

    local feval = function(x)
        if x ~= parameters then
            parameters:copy(x)
        end

        grad_parameters:zero()

        local f = 0
        for i = 1, num_inputs do
            local X_i = copy_example(inputs[i])
            local output = model:forward(X_i)
            local err = criterion:forward(output, targets[i])
            f = f + err

            local df_do = criterion:backward(output, targets[i])
            model:backward(inputs[i], df_do)
        end

        grad_parameters:div(num_inputs)
        f = f / num_inputs

        return f, grad_parameters
    end

    optim.adadelta(feval, parameters, {})
end

IN:input
	   6
 150
 300
[torch.LongStorage of size 3]

s1 in conv w/ filter size 3:input
	 150
 300
[torch.LongStorage of size 2]

s1 after conv w/ filter size 3:input
	   1
 150
 300
[torch.LongStorage of size 3]

s1 after conv pre relu w/ filter size 3:input
	   1
 148
   1
[torch.LongStorage of size 3]

s1 after relu pre max pooling w/ filter size 3:input
	   1
 148
   1
[torch.LongStorage of size 3]

s1 after max pooling w/ filter size 3:input
	 1
 1
 1
[torch.LongStorage of size 3]

s2 in conv w/ filter size 3:input
	 150
 300
[torch.LongStorage of size 2]

s2 after conv w/ filter size 3:input
	   1
 150
 300
[torch.LongStorage of size 3]

s2 after conv pre relu w/ filter size 3:input
	   1
 148
   1
[torch.LongStorage of size 3]

s2 after relu pre max pooling w/ filter size 3:input
	   1
 148
   1
[torch.LongStorage of size 3]

s2 after max pooling w/ filter size 3:input
	 1
 1
 1
[torch.LongStorage of size 3]

s1 in conv w/ filter size 4:input
	 150
 300
[torch.LongStorage of s

ongStorage of size 3]

s2 after conv w/ filter size 3:gradOutput
	   1
 150
 300
[torch.LongStorage of size 3]

s2 in conv w/ filter size 3:gradOutput
	 150
 300
[torch.LongStorage of size 2]

s1 after max pooling w/ filter size 4:gradOutput
	 1
 1
 1
[torch.LongStorage of size 3]

s1 after relu pre max pooling w/ filter size 4:gradOutput
	   1
 147
   1
[torch.LongStorage of size 3]

s1 after conv pre relu w/ filter size 4:gradOutput
	   1
 147
   1
[torch.LongStorage of size 3]

s1 after conv w/ filter size 4:gradOutput
	   1
 150
 300
[torch.LongStorage of size 3]

s1 in conv w/ filter size 4:gradOutput
	 150
 300
[torch.LongStorage of size 2]

s2 after max pooling w/ filter size 4:gradOutput
	 1
 1
 1
[torch.LongStorage of size 3]

s2 after relu pre max pooling w/ filter size 4:gradOutput
	   1
 147
   1
[torch.LongStorage of size 3]

s2 after conv pre relu w/ filter size 4:gradOutput
	   1
 147
   1
[torch.LongStorage of size 3]

s2 after conv w/ filter size 4:gradOutput
	   1
 15

t
	   1
 150
 300
[torch.LongStorage of size 3]

s1 after conv pre relu w/ filter size 5:input
	   1
 146
   1
[torch.LongStorage of size 3]

s1 after relu pre max pooling w/ filter size 5:input
	   1
 146
   1
[torch.LongStorage of size 3]

s1 after max pooling w/ filter size 5:input
	 1
 1
 1
[torch.LongStorage of size 3]

s2 in conv w/ filter size 5:input
	 150
 300
[torch.LongStorage of size 2]

s2 after conv w/ filter size 5:input
	   1
 150
 300
[torch.LongStorage of size 3]

s2 after conv pre relu w/ filter size 5:input
	   1
 146
   1
[torch.LongStorage of size 3]

s2 after relu pre max pooling w/ filter size 5:input
	   1
 146
   1
[torch.LongStorage of size 3]

s2 after max pooling w/ filter size 5:input
	 1
 1
 1
[torch.LongStorage of size 3]

AFTER p:input
	 6
 1
 1
[torch.LongStorage of size 3]

AFTER view:input
	 1
 6
[torch.LongStorage of size 2]

AFTER L2:input
	 1
 6
[torch.LongStorage of size 2]

AFTER dropout:input
	 1
 6
[torch.LongStorage of size 2]

AFTER linear:i

  1
 150
 300
[torch.LongStorage of size 3]

s2 in conv w/ filter size 5:gradOutput
	 150
 300
[torch.LongStorage of size 2]

IN:gradOutput
	   6
 150
 300
[torch.LongStorage of size 3]

 1
 8
[torch.LongStorage of size 2]

 8
[torch.LongStorage of size 1]

IN:input
	   6
 150
 300
[torch.LongStorage of size 3]

s1 in conv w/ filter size 3:input
	 150
 300
[torch.LongStorage of size 2]

s1 after conv w/ filter size 3:input
	   1
 150
 300
[torch.LongStorage of size 3]

s1 after conv pre relu w/ filter size 3:input
	   1
 148
   1
[torch.LongStorage of size 3]

s1 after relu pre max pooling w/ filter size 3:input
	   1
 148
   1
[torch.LongStorage of size 3]

s1 after max pooling w/ filter size 3:input
	 1
 1
 1
[torch.LongStorage of size 3]

s2 in conv w/ filter size 3:input
	 150
 300
[torch.LongStorage of size 2]

s2 after conv w/ filter size 3:input
	   1
 150
 300
[torch.LongStorage of size 3]

s2 after conv pre relu w/ filter size 3:input
	   1
 148
   1
[torch.LongStorage of size

 pooling w/ filter size 3:gradOutput
	 1
 1
 1
[torch.LongStorage of size 3]

s2 after relu pre max pooling w/ filter size 3:gradOutput
	   1
 148
   1
[torch.LongStorage of size 3]

s2 after conv pre relu w/ filter size 3:gradOutput
	   1
 148
   1
[torch.LongStorage of size 3]

s2 after conv w/ filter size 3:gradOutput
	   1
 150
 300
[torch.LongStorage of size 3]

s2 in conv w/ filter size 3:gradOutput
	 150
 300
[torch.LongStorage of size 2]

s1 after max pooling w/ filter size 4:gradOutput
	 1
 1
 1
[torch.LongStorage of size 3]

s1 after relu pre max pooling w/ filter size 4:gradOutput
	   1
 147
   1
[torch.LongStorage of size 3]

s1 after conv pre relu w/ filter size 4:gradOutput
	   1
 147
   1
[torch.LongStorage of size 3]

s1 after conv w/ filter size 4:gradOutput
	   1
 150
 300
[torch.LongStorage of size 3]

s1 in conv w/ filter size 4:gradOutput
	 150
 300
[torch.LongStorage of size 2]

s2 after max pooling w/ filter size 4:gradOutput
	 1
 1
 1
[torch.LongStorage of size 

...

nv pre relu w/ filter size 4:gradOutput
	   1
 147
   1
[torch.LongStorage of size 3]

s1 after conv w/ filter size 4:gradOutput
	   1
 150
 300
[torch.LongStorage of size 3]

s1 in conv w/ filter size 4:gradOutput
	 150
 300
[torch.LongStorage of size 2]

s2 after max pooling w/ filter size 4:gradOutput
	 1
 1
 1
[torch.LongStorage of size 3]

s2 after relu pre max pooling w/ filter size 4:gradOutput
	   1
 147
   1
[torch.LongStorage of size 3]

s2 after conv pre relu w/ filter size 4:gradOutput
	   1
 147
   1
[torch.LongStorage of size 3]

s2 after conv w/ filter size 4:gradOutput
	   1
 150
 300
[torch.LongStorage of size 3]

s2 in conv w/ filter size 4:gradOutput
	 150
 300
[torch.LongStorage of size 2]

s1 after max pooling w/ filter size 5:gradOutput
	 1
 1
 1
[torch.LongStorage of size 3]

s1 after relu pre max pooling w/ filter size 5:gradOutput
	   1
 146
   1
[torch.LongStorage of size 3]

s1 after conv pre relu w/ filter size 5:gradOutput
	   1
 146
   1
[torch.LongStorage

2]

s2 after conv w/ filter size 5:input
	   1
 150
 300
[torch.LongStorage of size 3]

s2 after conv pre relu w/ filter size 5:input
	   1
 146
   1
[torch.LongStorage of size 3]

s2 after relu pre max pooling w/ filter size 5:input
	   1
 146
   1
[torch.LongStorage of size 3]

s2 after max pooling w/ filter size 5:input
	 1
 1
 1
[torch.LongStorage of size 3]

AFTER p:input
	 6
 1
 1
[torch.LongStorage of size 3]

AFTER view:input
	 1
 6
[torch.LongStorage of size 2]

AFTER L2:input
	 1
 6
[torch.LongStorage of size 2]

AFTER dropout:input
	 1
 6
[torch.LongStorage of size 2]

AFTER linear:input
	 1
 8
[torch.LongStorage of size 2]

 0.5502  0.4079  0.4404  0.6101  0.4406  0.4761  0.4492  0.4932
[torch.DoubleTensor of size 1x8]

AFTER linear:gradOutput
	 1
 8
[torch.LongStorage of size 2]

AFTER dropout:gradOutput
	 1
 6
[torch.LongStorage of size 2]

AFTER L2:gradOutput
	 1
 6
[torch.LongStorage of size 2]

AFTER view:gradOutput
	 1
 6
[torch.LongStorage of size 2]

AFTER p:gradOut

conv w/ filter size 3:input
	   1
 150
 300
[torch.LongStorage of size 3]

s1 after conv pre relu w/ filter size 3:input
	   1
 148
   1
[torch.LongStorage of size 3]

s1 after relu pre max pooling w/ filter size 3:input
	   1
 148
   1
[torch.LongStorage of size 3]

s1 after max pooling w/ filter size 3:input
	 1
 1
 1
[torch.LongStorage of size 3]

s2 in conv w/ filter size 3:input
	 150
 300
[torch.LongStorage of size 2]

s2 after conv w/ filter size 3:input
	   1
 150
 300
[torch.LongStorage of size 3]

s2 after conv pre relu w/ filter size 3:input
	   1
 148
   1
[torch.LongStorage of size 3]

s2 after relu pre max pooling w/ filter size 3:input
	   1
 148
   1
[torch.LongStorage of size 3]

s2 after max pooling w/ filter size 3:input
	 1
 1
 1
[torch.LongStorage of size 3]

s1 in conv w/ filter size 4:input
	 150
 300
[torch.LongStorage of size 2]

s1 after conv w/ filter size 4:input
	   1
 150
 300
[torch.LongStorage of size 3]

s1 after conv pre relu w/ filter size 4:input
	  

bad argument #2 to '?' (out of range at /Users/nexus/torch/pkg/torch/generic/Tensor.c:880)
stack traceback:
	[C]: at 0x05842ae0
	[C]: in function '__index'
	[string "for i = 1, no_examples, batch_size do..."]:14: in function 'opfunc'
	/Users/nexus/torch/install/share/lua/5.1/optim/adadelta.lua:29: in function 'adadelta'
	[string "for i = 1, no_examples, batch_size do..."]:34: in main chunk
	[C]: in function 'xpcall'
	/Users/nexus/torch/install/share/lua/5.1/itorch/main.lua:179: in function </Users/nexus/torch/install/share/lua/5.1/itorch/main.lua:143>
	/Users/nexus/torch/install/share/lua/5.1/lzmq/poller.lua:75: in function 'poll'
	/Users/nexus/torch/install/share/lua/5.1/lzmq/impl/loop.lua:307: in function 'poll'
	/Users/nexus/torch/install/share/lua/5.1/lzmq/impl/loop.lua:325: in function 'sleep_ex'
	/Users/nexus/torch/install/share/lua/5.1/lzmq/impl/loop.lua:370: in function 'start'
	/Users/nexus/torch/install/share/lua/5.1/itorch/main.lua:350: in main chunk
	[C]: in function 'requi

radOutput
	 150
 300
[torch.LongStorage of size 2]

s1 after max pooling w/ filter size 4:gradOutput
	 1
 1
 1
[torch.LongStorage of size 3]

s1 after relu pre max pooling w/ filter size 4:gradOutput
	   1
 147
   1
[torch.LongStorage of size 3]

s1 after conv pre relu w/ filter size 4:gradOutput
	   1
 147
   1
[torch.LongStorage of size 3]

s1 after conv w/ filter size 4:gradOutput
	   1
 150
 300
[torch.LongStorage of size 3]

s1 in conv w/ filter size 4:gradOutput
	 150
 300
[torch.LongStorage of size 2]

s2 after max pooling w/ filter size 4:gradOutput
	 1
 1
 1
[torch.LongStorage of size 3]

s2 after relu pre max pooling w/ filter size 4:gradOutput
	   1
 147
   1
[torch.LongStorage of size 3]

s2 after conv pre relu w/ filter size 4:gradOutput
	   1
 147
   1
[torch.LongStorage of size 3]

s2 after conv w/ filter size 4:gradOutput
	   1
 150
 300
[torch.LongStorage of size 3]

s2 in conv w/ filter size 4:gradOutput
	 150
 300
[torch.LongStorage of size 2]

s1 after max pooling w

## Evaluation

In [134]:
confusion = optim.ConfusionMatrix(classes)
model:evaluate()

In [ ]:

-- confusion:add(output:reshape(8), targets[i]:reshape(8))

## Sources
* **(Kim, 2014):** Convolutional Neural Networks for Sentence Classification by Yoon Kim
* **(Zhang and Wallace, 2015):** A Sensitivity Analysis of (and Practitioners' Guide to) Convolutional Neural Networks for Sentence Classification by Ye Zhang, Byron Wallace